# Purpose
I want to be able to figure out which parkruns I can in future weeks and I want to know the estimated number of events (for the event number).

To do this, I plan to scrape the UK Parkrun Wikipedia page, which has a list of UK Parkruns segmented by region. I want a complete list with predicted event number.

In [14]:
import numpy as np
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

In [10]:
response = requests.get(
	url="https://en.wikipedia.org/wiki/List_of_Parkruns_in_the_United_Kingdom",
)
print(response.status_code)

200


In [13]:
response = requests.get(
	url="https://en.wikipedia.org/wiki/List_of_Parkruns_in_the_United_Kingdom",
)

soup = BeautifulSoup(response.content, 'html.parser')

title = soup.find(id="firstHeading")
print(title.string)

List of Parkruns in the United Kingdom


In [88]:
location = soup.find("span", {"class":"mw-headline"})
print(location.string)


locations = soup.find_all("span", {"class":"mw-headline"})
loc_container = np.empty(len(locations), dtype=object)


for i in range(len(locations)):
    loc_container[i] = locations[i].text


Channel Islands


In [92]:
Location_list = loc_container[:13]

In [67]:
# Channel islands

channel = soup.find('table',{'class':"wikitable"})

#print(channel)

In [24]:
df = pd.read_html(str(channel))
df = pd.DataFrame(df[0])
print(df.head())

channel_data = df.drop(["Notes"], axis=1)

channel_data.head()


                   Name               Location   Started on  \
0  Guernsey Parkrun[10]   L'Ancresse, Guernsey   9 Apr 2016   
1    Jersey Parkrun[11]  Saint Brelade, Jersey  26 Sep 2015   

                                               Notes  
0  A coastal two-lap course over mixed terrain an...  
1  Starts and finishes at Les Quennevais Sports C...  


,Name,Location,Started on
0,Guernsey Parkrun[10],"L'Ancresse, Guernsey",9 Apr 2016
1,Jersey Parkrun[11],"Saint Brelade, Jersey",26 Sep 2015


In [240]:
# All

all = soup.find_all('table',{'class':"wikitable"})

df = pd.read_html(str(all))



for i in range(len(df)):
    
    Location_list[i] = pd.DataFrame(df[i])


# df = df.drop(["Notes"])

# channel = df[0]
# eastmid = df[1]
# easteng = df[2]


# df = pd.DataFrame(df[0])
# print(df.head())

# data = df.drop(["Notes"], axis=1)

# data.head()

In [241]:
for i in range(len(Location_list)):
    if i>0:
        old = str(Location_list[i].columns[3])
        Location_list[i] = Location_list[i].rename(columns={old:"Date"})

In [242]:
data = Location_list[1]

for i in range(len(Location_list)):
    if i>1:
        data = pd.merge(data, Location_list[i], how='outer')

In [243]:
data.to_csv("parkrun1.csv")

In [244]:
from datetime import datetime, timedelta
d_today = datetime.today().strftime('%Y-%m-%d')

d_today = datetime.today()

d_today

data['Date'] = pd.to_datetime(data['Date'])

datetime.datetime(2022, 11, 6, 17, 56, 54, 303730)

In [271]:
event_no = (d_today - data['Date'])/7

event_no = event_no.dt.days

data['Event Number'] = event_no

In [274]:
data = data.drop(["Notes","Date"], axis=1)

In [275]:
data.to_csv("parkrun2.csv")